In [1]:
import os
import time
import random
import re
import pandas as pd
import numpy as np

In [2]:
import jieba

In [3]:
import pickle

In [4]:
def dump_pickle(obj,fname):
    with open(fname,'wb') as file:
        pickle.dump(obj,file)
def load_pickle(fname):
    return pickle.load(open(fname,'rb'))

In [5]:
claim_df = pd.read_pickle('../data/patent_data/processed/claim_df_seg_0630.pkl')

In [27]:
patent_text = [''.join(i) for i in claim_df.seg_text.tolist()]

# 关键词抽取

使用textrank4zh包里的相关内容

In [8]:
from textrank4zh import TextRank4Keyword,TextRank4Sentence

tr4w = TextRank4Keyword()
def topkword(text,topk=20):
    tr4w.analyze(text,lower=True,window=2)
    return tr4w.keywords[:topk]

In [ ]:
claim_df['keywords'] = claim_df.text.apply(lambda x:topkword(x))

In [9]:
start_point = 96443

In [ ]:
with open('../data/patent_data/processed/claim_kword.txt','a',encoding='utf-8') as file:
    for num,_ in enumerate(claim_df.itertuples()):
        if num <start_point:
            continue
        if num % 1000==0:
            print('[{0}] Num:{1}'.format(time.ctime(),num))
        kw = topkword(claim_df.loc[num,'text'])
        kw_line = ['{0}:{1}'.format(i['word'],i['weight']) for i in kw]
        kw_line = '|'.join(kw_line)
        file.write(kw_line+'\n')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.962 seconds.
Prefix dict has been built succesfully.


# 合并关键词

In [6]:
claim_df

,id,pub_id,info_id,text,seg_text
0,0,CN103257847B,claim-info-CN2013,一种处理器，包括：位于单个芯片封装内的多个处理核，其中每个处理核进一步包括：指令取出逻辑单元...,"[一种, 处理器, ，, 包括, ：, 位于, 单个, 芯片, 封装, 内, 的, 多个, ..."
1,1,CN103170050B,claim-info-CN2013,一种导管组件，包括：导管本体，所述导管本体限定至少一第一内腔和第二内腔，所述导管本体包括远侧...,"[一种, 导管, 组件, ，, 包括, ：, 导管, 本体, ，, 所述, 导管, 本体, ..."
2,2,CN103414917B,claim-info-CN2013,一种用于从远程地点上传媒体内容的虚拟宽带传送单元，包括：多个调制解调器管理器，每个调制解调器...,"[一种, 用于, 从, 远程, 地点, 上传, 媒体, 内容, 的, 虚拟, 宽带, 传送,..."
3,3,CN103199951B,claim-info-CN2013,一种通信系统中的移动台，所述移动台包括：一种装置，所述装置用于向基站发送帧，其中，第一Zad...,"[一种, 通信, 系统, 中, 的, 移动, 台, ，, 所述, 移动, 台, 包括, ：,..."
4,4,CN103527484B,claim-info-CN2013,一种回转式压缩机，其压缩作为制冷剂的HFO-1234yf和R32或R41的混合物，其特征在于...,"[一种, 回转式, 压缩机, ，, 其, 压缩, 作为, 制冷剂, 的, HFO, -, 1..."
5,5,CN103498666B,claim-info-CN2013,三维天线，包括：支撑结构，包括第一面板、第二面板和第三面板；支撑在第一面板中且界定出有孔区域...,"[三维, 天线, ，, 包括, ：, 支撑, 结构, ，, 包括, 第一, 面板, 、, 第..."
6,6,CN103183814B,claim-info-CN2013,一种来自生物质资源的聚酯，其主要重复单元为二羧酸单元和二醇单元，其中，所述二羧酸单元是具有脂...,"[一种, 来自, 生物质, 资源, 的, 聚酯, ，, 其, 主要, 重复, 单元, 为, ..."
7,7,CN103367654B,claim-info-CN2013,一种发光元件，包括：第一电极；第二电极；和包含第一电极和第二电极之间的发光物质的第一层，其中...,"[一种, 发光, 元件, ，, 包括, ：, 第一, 电极, ；, 第二, 电极, ；, 和..."
8,8,CN103624740B,claim-info-CN2013,一种电动操纵的手持挤压机的工作方法，其中，当操纵开关时，一个或多个压钳便从起始位置运动到闭合...,"[一种, 电动, 操纵, 的, 手持, 挤压机, 的, 工作, 方法, ，, 其中, ，, ..."
9,9,CN103498704B,claim-info-CN2013,一种双向流冲击式涡轮装置，所述涡轮装置与经过该涡轮装置的双向反向流一起使用，所述涡轮装置具有...,"[一种, 双向, 流, 冲击, 式, 涡轮, 装置, ，, 所述, 涡轮, 装置, 与, 经..."


In [7]:
kword = [i.replace('\n','').split('|') for i in open('../data/patent_data/processed/claim_kword.txt')]

In [9]:
kword = [[p.split(':')[0] for p in i]for i in kword]

In [11]:
claim_df['kword_tr4w'] = [' '.join(i) for i in kword]

In [12]:
claim_df

,id,pub_id,info_id,text,seg_text,kword_tr4w
0,0,CN103257847B,claim-info-CN2013,一种处理器，包括：位于单个芯片封装内的多个处理核，其中每个处理核进一步包括：指令取出逻辑单元...,"[一种, 处理器, ，, 包括, ：, 位于, 单个, 芯片, 封装, 内, 的, 多个, ...",值 存储 用于 转换 向量 指令 寄存器 包括 数目 浮点 整数 特征 芯片 封装 处理 核...
1,1,CN103170050B,claim-info-CN2013,一种导管组件，包括：导管本体，所述导管本体限定至少一第一内腔和第二内腔，所述导管本体包括远侧...,"[一种, 导管, 组件, ，, 包括, ：, 导管, 本体, ，, 所述, 导管, 本体, ...",开口 限定 包括 本体 部分 横向 结构 导管 流体 内腔 流动 相等 再循环 弯曲 延伸 ...
2,2,CN103414917B,claim-info-CN2013,一种用于从远程地点上传媒体内容的虚拟宽带传送单元，包括：多个调制解调器管理器，每个调制解调器...,"[一种, 用于, 从, 远程, 地点, 上传, 媒体, 内容, 的, 虚拟, 宽带, 传送,...",调制解调器 用于 包括 分组 网络 进行 上传 视频 传输 数据 接收 分析 连接 性能 质...
3,3,CN103199951B,claim-info-CN2013,一种通信系统中的移动台，所述移动台包括：一种装置，所述装置用于向基站发送帧，其中，第一Zad...,"[一种, 通信, 系统, 中, 的, 移动, 台, ，, 所述, 移动, 台, 包括, ：,...",序列 包括 基站 表示 exp 用于 chu 分配 整数 移动 权利 要求 参数 值 通信 ...
4,4,CN103527484B,claim-info-CN2013,一种回转式压缩机，其压缩作为制冷剂的HFO-1234yf和R32或R41的混合物，其特征在于...,"[一种, 回转式, 压缩机, ，, 其, 压缩, 作为, 制冷剂, 的, HFO, -, 1...",构成 部 材料 具有 元件 滑动 偏心 叶片 压缩 处理 压缩机 dlc 曲轴 wc 柱塞 ...
5,5,CN103498666B,claim-info-CN2013,三维天线，包括：支撑结构，包括第一面板、第二面板和第三面板；支撑在第一面板中且界定出有孔区域...,"[三维, 天线, ，, 包括, ：, 支撑, 结构, ，, 包括, 第一, 面板, 、, 第...",包括 线圈 矩形 孔 具有 支撑 天线 权利 要求 正交 定向 出有 轻质 磁 结构 区域 ...
6,6,CN103183814B,claim-info-CN2013,一种来自生物质资源的聚酯，其主要重复单元为二羧酸单元和二醇单元，其中，所述二羧酸单元是具有脂...,"[一种, 来自, 生物质, 资源, 的, 聚酯, ，, 其, 主要, 重复, 单元, 为, ...",聚酯 单元 权利 资源 树脂 二羧酸 官能 方法 质量 反应 原料 原子 和二醇 含量 化合...
7,7,CN103367654B,claim-info-CN2013,一种发光元件，包括：第一电极；第二电极；和包含第一电极和第二电极之间的发光物质的第一层，其中...,"[一种, 发光, 元件, ，, 包括, ：, 第一, 电极, ；, 第二, 电极, ；, 和...",酞菁 氧化 包含 包括 发光 元件 具有 氧化锌 区域 光 材料 酰亚胺 萘四 氧化钨 低于...
8,8,CN103624740B,claim-info-CN2013,一种电动操纵的手持挤压机的工作方法，其中，当操纵开关时，一个或多个压钳便从起始位置运动到闭合...,"[一种, 电动, 操纵, 的, 手持, 挤压机, 的, 工作, 方法, ，, 其中, ，, ...",压力 挤压 借助 传感器 特征 操纵 达到 自动 检测 规定 位置 运动 设有 测量 检验 ...
9,9,CN103498704B,claim-info-CN2013,一种双向流冲击式涡轮装置，所述涡轮装置与经过该涡轮装置的双向反向流一起使用，所述涡轮装置具有...,"[一种, 双向, 流, 冲击, 式, 涡轮, 装置, ，, 所述, 涡轮, 装置, 与, 经...",轴向 轮叶 限定 转向 包括 叶片 流体 具有 流 导向 装置 转子 轴线 角度 流动 圆弧...


In [13]:
del claim_df['text']
del claim_df['seg_text']

In [14]:
claim_df.to_pickle('../data/patent_data/processed/claim_kword_df_tr4w.pkl')